In [30]:
import sys
import os

%matplotlib inline 
from __future__ import division
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from scipy.stats import chisquare, chi2_contingency

BASE_COUNT = 50

def readData(file):
    fin = open(file, 'r')
    data = fin.read()
    fin.close()

    data = list(map(lambda s: s.strip(), data))  # remove '/n'
    data = list(filter(None, data))  # remove empty string
    
    return data

def detectChange_Categoral(threshold, WINDOW_LEN, MOVING_INTERVAL, data):
    baseline = data[0:BASE_COUNT]

    variables = list(set(baseline))
    
    freq_old = np.zeros(len(variables))
    freq = np.zeros(len(variables))

    k = 0
    while (k + 2 * WINDOW_LEN < len(data)):
        
        p = calculateP(variables, k, data, WINDOW_LEN)

        
        if(k + WINDOW_LEN > 50 and p < threshold):
            start = k + WINDOW_LEN            
            for i in range(WINDOW_LEN):
                k_track = start - WINDOW_LEN - i
                p_track = calculateP(variables, k_track, data, WINDOW_LEN)
                if(p_track > 0.05):
                    end = k_track + 2*WINDOW_LEN
                    break

            if (end - WINDOW_LEN/2 > start):
                position = int(end - WINDOW_LEN/2)
            else:
                position = int((start + end)/2)
            
            return position

        k = k + MOVING_INTERVAL        
    
    return -1

    
    
def calculateP(variables, k, data, WINDOW_LEN):
    
    freq_old = np.zeros(len(variables))
    freq = np.zeros(len(variables))

    for i in range(len(variables)):
        sample = data[k:k+WINDOW_LEN]
        freq_old[i] = sample.count(variables[i])

        sample = data[k+WINDOW_LEN : k+2*WINDOW_LEN]
        freq[i] = sample.count(variables[i])

    if (sum(freq==0)>0 or sum(freq_old==0)>0):
        chi = chisquare(freq, freq_old)
    else:
        chi = chi2_contingency([freq,freq_old], correction=True)

    if (len(variables)==2):
        chi = chisquare(freq, freq_old)
    
    p = chi[1]
        
    return p

In [32]:
files = ['sampleData/posTriple_175.txt', 'sampleData/posTripleDouble_175.txt', 'sampleData/posTriple2_160.txt', 
        'sampleData/posBinary_150.txt', 'sampleData/negTriple.txt', 'sampleData/negBinary.txt',
        'sampleData/negWithRare.txt']

WINDOW_LEN = 40
MOVING_INTERVAL = 5

# 40, 5, 0.0005 works good so far.

# files = ['sampleData/posBinary_150.txt']


for i in range(len(files)):
    file = files[i]
    data = readData(file)

    print(file, detectChange_Categoral(0.0005, 40, 1, data))

sampleData/posTriple_175.txt 163
sampleData/posTripleDouble_175.txt 176
sampleData/posTriple2_160.txt 161
sampleData/posBinary_150.txt 143
sampleData/negTriple.txt -1
sampleData/negBinary.txt -1
sampleData/negWithRare.txt -1


In [49]:
testFile = np.random.choice(['i','j','k'], size=139, replace=True, p=[0.1,0.8,0.1])
testFile = np.append(testFile,np.random.choice(['i','j','k'], size=300, replace=True, p=[0.5,0.4,0.1]))

f = open('test_Yan/bin_posTriple2_140.txt', 'w+') #open file for write and read
for s in testFile:
    f.write(str(s) + "\n")
f.close()
